In [ ]:
!pip install --upgrade git+https://github.com/EmGarr/od.git
# Useful for tensorboard
!pip install --upgrade grpcio

In [ ]:
#%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Download COCO/2017

Download and preprocess COCO/2017 to the following format (required by od networks):

```python
dataset = {
        'images' : A tensor of float32 and shape [1, height, widht, 3],
        'images_info': A tensor of float32 and shape [1, 2] ,
        'bbox': A tensor of float32 and shape [1, num_boxes, 4],
        'labels': A tensor of int32 and shape [1, num_boxes],
        'num_boxes': A tensor of int32 and shape [1, 1],
        'weights': A tensor of float32 and shape [1, num_boxes]
    }
```

In [ ]:
from od.dataset.preprocessing import resize_to_min_dim
from od.core.standard_fields import DatasetField, BoxField
from od.core.box_ops import compute_area

def filter_crowded_boxes(boxes, crowd):
    """Coco has boxes flagged as crowded which are not used during the training.
    This function will discard them.
    """
    ind_uncrowded_boxes = tf.where(tf.equal(crowd, False))
    return tf.gather_nd(boxes, ind_uncrowded_boxes)


def filter_bad_area(boxes):
    """Few boxes have an area equal to 0 it will clean them
    """
    area = compute_area(boxes)
    return tf.gather_nd(boxes, tf.where(area > 0))


def preprocess(inputs):
    """This operations performs a classical preprocessing operations for localization datasets:

    - COCO
    - Pascal Voc

    You can download easily those dataset using [tensorflow dataset](https://www.tensorflow.org/datasets/catalog/overview).

    Argument:

    - *inputs*: It can be either a [FeaturesDict](https://www.tensorflow.org/datasets/api_docs/python/tfds/features/FeaturesDict) or a dict.
    but it should have the following structures.

    ```python
    inputs = FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'objects': Sequence({
            'area': Tensor(shape=(), dtype=tf.int64), # area
            'bbox': BBoxFeature(shape=(4,), dtype=tf.float32), # The values are between 0 and 1
            'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=80),
        }),
    })
    ```

    Returns:

    - *inputs*:
        1. image: A 3D tensor of float32 and shape [None, None, 3]
        2. image_information: A 1D tensor of float32 and shape [(height, width),]. It contains the shape
        of the image without any padding. It can be usefull if it followed by a `padded_batch` operations.
        The models needs those information in order to clip the boxes to the proper dimension.
    - *inputs*: A dict with the following information

    ```
    inputs = {
        BoxField.BOXES: A tensor of shape [num_boxes, (y1, x1, y2, x2)] and resized to the image shape
        BoxField.LABELS: A tensor of shape [num_boxes, ]
        BoxField.NUM_BOXES: A tensor of shape (). It is usefull to unpad the data in case of a batched training
    }
    ```
    """
    image = resize_to_min_dim(inputs['image'], 800.0, 1300.0)
    image_information = tf.cast(tf.shape(image)[:2], dtype=tf.float32)
    boxes = inputs['objects'][BoxField.BOXES] * tf.tile(tf.expand_dims(image_information, axis=0),
                                                        [1, 2])
    boxes = filter_crowded_boxes(boxes, inputs['objects']['is_crowd'])
    boxes = filter_bad_area(boxes)
    
    return {
        DatasetField.IMAGES: image,
        DatasetField.IMAGES_INFO: image_information,
        BoxField.BOXES: boxes,
        BoxField.LABELS: inputs['objects'][BoxField.LABELS],
        BoxField.NUM_BOXES: tf.shape(inputs['objects'][BoxField.LABELS]),
        BoxField.WEIGHTS: tf.fill(tf.shape(inputs['objects'][BoxField.LABELS]), 1.0)
    }


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from od.dataset.preprocessing import expand_dims_for_single_batch

ds_train, ds_info = tfds.load(name="coco/2017", split="train", shuffle_files=True, with_info=True)
ds_train = ds_train.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.map(expand_dims_for_single_batch, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = tfds.load(name="coco/2017", split="validation", shuffle_files=False)
ds_test = ds_test.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(expand_dims_for_single_batch, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
ds_info

# Load and train the network


In [ ]:
from od.model.faster_rcnn import build_fpn_resnet50_faster_rcnn
from od.core.standard_fields import BoxField
from od.core.learning_rate_schedule import WarmupLearningRateScheduler
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.mixed_precision import experimental as mixed_precision

# Number of classes of COCO
classes = ds_info.features['objects']['label'].names
num_classes = len(classes)
batch_size = 1

model_faster_rcnn = build_fpn_resnet50_faster_rcnn(num_classes, batch_size)
base_lr = 0.02
optimizer = tf.keras.optimizers.SGD(learning_rate=base_lr)
model_faster_rcnn.compile(optimizer=optimizer, loss=None)
callbacks = [WarmupLearningRateScheduler(base_lr, 1, epochs=[8, 10], init_lr=0.0001), TensorBoard(), ModelCheckpoint('.checkpoints/')]

model_faster_rcnn.fit(ds_train, validation_data=ds_test, epochs=11, callbacks=callbacks)

In [ ]:
# Save the weights for the serving
model_faster_rcnn.save_weights('final_weights.h5')

# Tensorboard

In [ ]:
# Load TENSORBOARD
%load_ext tensorboard
# Start TENSORBOARD
%tensorboard --logdir logs

# Export for inference

When you are in training mode all the ground_truths are used as inputs:
- BoxField.BOXES
- BoxField.LABELS
- BoxField.NUM_BOXES
- BoxField.WEIGHTS

We want to remove those for the serving.

The first step is to rebuild the graph in inference mode. Reload the `final_weights.h5` and save using the save method from `tf.keras.Model`.


In [ ]:
from od.model.faster_rcnn import build_fpn_resnet50_faster_rcnn

model_faster_rcnn_inference = build_fpn_resnet50_faster_rcnn(num_classes, None, training=False)
model_faster_rcnn_inference.load_weights('final_weights.h5')
model_faster_rcnn_inference.save('serving_model', include_optimizer=False)